In [1]:
import pandas as pd
import docx
import redis
import pyarrow as pa

In [2]:
def parse_document(doc_path, skill_path, tier_start=-1):
    doc = docx.Document(doc_path)
    paras = [x.text for x in doc.paragraphs]
    for i in range(len(paras)):
        if paras[i] == ' ':
            paras[i] = ''
    particular_value = ''
    result = []
    temp_list = []
    for i in paras:
        if i == particular_value:
            temp_list.append(i)
            result.append(temp_list)
            temp_list = []
        else:
            temp_list.append(i)
    result.append(temp_list)
    df = pd.DataFrame({'text':result})
    tier_list = []
    ability_list = []
    tier = tier_start
    for _, row in df.iterrows():
        data = row['text']
        if len(data) > 1:
            if data[0].split(' ')[0] == 'Tier':
                tier += 1
                data.pop(0)
            tier_list.append(tier)
            ability_list.append(data)
    ability_df = pd.DataFrame({'Ability':ability_list, 'Tier':tier_list})
    ability_df['Path'] = skill_path
    ability_df['Skill Name'] = ability_df.Ability.apply(lambda x:x[0].split(':')[0])
    ability_df['Description'] = ability_df.Ability.apply(lambda x:x[0].split(':')[1])
    pr = []
    lim = []
    preq = []
    for _ , row in ability_df.iterrows():
        for sublist in row['Ability']:
            if 'Phys Rep' in sublist:
                pr.append(sublist.split(':')[1])
            if 'Limitations' in sublist:
                lim.append(sublist.split(':')[1])
            if 'Prerequisite' in sublist:
                preq.append(sublist.split(':')[1])
        if 'Phys Rep' not in str(row.Ability):
            pr.append(None)
        if 'Limitations' not in str(row.Ability):
            lim.append(None)
        if 'Prerequisite' not in str(row.Ability):
            preq.append(None)
    ability_df['Phys Rep'] = pr
    ability_df['Limitations'] = lim
    ability_df['Prerequisite'] = preq
    ability_df = ability_df.map(lambda x: x.strip() if isinstance(x, str) else x)
    return ability_df[['Skill Name', 'Description', 'Path', 'Tier', 'Limitations', 'Phys Rep', 'Prerequisite']]

In [3]:
warrior_df = parse_document("The Warrior's Path.docx", 'Warrior')
rogue_df = parse_document("The Rogue's Path.docx", 'Rogue')
healer_df = parse_document("The Healer's Path.docx", 'Healer')
mage_df = parse_document("The Mage's Path.docx", 'Mage')
bard_df = parse_document("The Bard's Path.docx", 'Bard', tier_start=0)

In [4]:
skills = pd.concat([warrior_df,rogue_df,healer_df,mage_df,bard_df])

In [5]:
skills['Spell'] = skills['Description'].apply(lambda x: x.startswith(('(Spell)', '(Combat Magic)')))

In [6]:
skills

,Skill Name,Description,Path,Tier,Limitations,Phys Rep,Prerequisite,Spell
0,Basic Weapon Proficiency,Players learn the basic combat and safety rule...,Warrior,0,None,None,None,False
1,Armor Proficiency,Players learn about the armor that they wear a...,Warrior,0,None,None,None,False
2,Kindle Flame/Torch,Player gains proficiency at creating normal fi...,Warrior,0,10’ radius,Optional (larp safe electronically lighted tor...,None,False
3,Shield Control,Players learn the art of Shield Control and ma...,Warrior,0,None,None,None,False
4,Advanced Weapon Training,Study and become proficient in the use of two-...,Warrior,1,None,None,None,False
...,...,...,...,...,...,...,...,...
8,Curative Works,The Bard is able to channel more powerful heal...,Bard,4,Can only be used once per day with an addition...,"A song, lasting 30 seconds, played for specifi...",Healing Song,False
9,Song of Sight,The Bard is able to play a melody that resonat...,Bard,4,Can only be used once per day with an addition...,"A song, lasting 30 seconds, played towards the...",None,False
10,Group Inspiration,The Bard greatly projects their voice and thei...,Bard,5,The additional armor point only lasts only as ...,A 60 second song intro is played to a group of...,Party Inspiration,False
11,Song of Life,The Bard can fully restore a mortally wounded ...,Bard,6,Can only be used once a day plus one for every...,"A song of at least 2 minutes, played to a mort...",Curative Works,False


In [7]:
skills[skills.Path == 'Mage']

,Skill Name,Description,Path,Tier,Limitations,Phys Rep,Prerequisite,Spell
0,Read/Write Arcana,Player can decipher magic runes and read magic...,Mage,0,"Study, research, and quests may be needed to u...",Rune cipher in spellbook (Elder Futhark is the...,None,False
1,Light,(Spell) Create light equal to 1 torch. Can cou...,Mage,0,10’ radius,10 word spell in spellbook. Optional (lighted ...,None,True
2,Darkness,(Spell) Counters magical light. May be used to...,Mage,0,10’ radius,"10 word spell in spellbook. Optional, black cl...",None,True
3,Sense Arcana,"Can sense if an object has arcane properties, ...",Mage,0,"Single person or object, may be used 3x per da...",10 word spell in spellbook.,None,False
4,Basic Weapon Proficiency,Players learn the basic combat and safety rule...,Mage,0,None,None,None,False
5,Kindle Flame/Torch,Player gains proficiency at creating normal fi...,Mage,0,10’ radius,LARP safe light source.,None,False
6,Minor Repair,(Spell) Repairs a single small normal object 1...,Mage,1,"Single person or object/touch, may be used 3x ...","25 word spell in spellbook, and roleplay.",None,True
7,Divine Arcana,(Spell) Once an object is defined as magical t...,Mage,1,1x per Mage Tier per day per day.,"25 word spell in spellbook, and roleplay.",Sense Arcana,True
8,Arcane Armor,(Spell) Absorbs the first 1 point of damage ta...,Mage,1,May be used 1x per day +1 per Mage Tier above ...,"25 word spell in spellbook, red belt flag with...",None,True
9,Ward Location,(Spell) Areas marked cannot be entered or exit...,Mage,1,"1x per Tier per day, 10’ radius per tier, a Ma...","25 word spell in spellbook, and roleplay. Yell...",None,True


In [8]:
skills.to_excel('Skills_Table.xlsx', index=False)

In [9]:
skills.to_csv('Skills_Table.csv', index=False)

In [10]:
from math import floor, sqrt
def tier(events):
    return floor((sqrt(8*events)-1)/2)

In [11]:
skills[skills['Skill Name']=='Basic Weapon Proficiency']

,Skill Name,Description,Path,Tier,Limitations,Phys Rep,Prerequisite,Spell
0,Basic Weapon Proficiency,Players learn the basic combat and safety rule...,Warrior,0,None,None,None,False
3,Basic Weapon Proficiency,Players learn the basic combat and safety rule...,Rogue,0,None,None,None,False
5,Basic Weapon Proficiency,Players learn the basic combat and safety rule...,Healer,0,None,None,None,False
4,Basic Weapon Proficiency,Players learn the basic combat and safety rule...,Mage,0,None,None,None,False
